In [1]:
cvox.PortSetupcvox.PortSetupimport org.apache.spark.sql.types.DoubleType
import org.apache.spark.sql.types.StringType
import org.apache.spark.sql.types.DecimalType
import org.apache.spark.sql.types.LongType
import org.apache.spark.sql.functions.regexp_replace
import org.apache.spark.sql.functions.monotonically_increasing_id
import org.apache.spark.ml.feature.PCA
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.sql.functions.lit
import org.apache.spark.ml.clustering.KMeans
import org.apache.spark.sql.types.StructType
import org.apache.spark.sql.types.StructField
import org.apache.spark.ml.feature.Normalizercvox.MathJaxPortSetup

In [2]:
def setHadoopConfig(name: String) = {
    val prefix = "fs.swift2d.service." + name
    sc.hadoopConfiguration.set(prefix + ".auth.url", "https://identity.open.softlayer.com" + "/v3/auth/tokens")
    sc.hadoopConfiguration.set(prefix + ".auth.endpoint.prefix","endpoints")
    sc.hadoopConfiguration.set(prefix + ".tenant", "a9fb4d478e3d40a8bbd54c5a2ecf25a3")
    sc.hadoopConfiguration.set(prefix + ".username", "6a4cc8251c1940179a6cccc9098a15e0")
    sc.hadoopConfiguration.set(prefix + ".password", "kDTcKA2H(3eo5.G0")
    sc.hadoopConfiguration.setInt(prefix + ".http.port", 8080)
    sc.hadoopConfiguration.set(prefix + ".region", "dallas")
    sc.hadoopConfiguration.setBoolean(prefix + ".public", false)
}

val name = "keystone"
setHadoopConfig(name)

In [3]:
/*val caseNumbers = Array(2, 3, 4, 5, 8, 9, 10, 11, 12)*/
/*don't work:
3,4,5
IllegalArgumentException: requirement failed: Decimal precision 6 exceeds max precision 5
do work:
1,2,8,9,10,11,12
*/
/* load in the first file to append (union) to */
var cases = spark.read.option("header",false).
    option("inferSchema",true).
    option("nanValue","0").
    option("nullValue","0").
    option("positiveInf","0").
    option("negativeInf","0").
    /*schema(schema).*/
    csv("swift2d://MGH." + name + "/csv/csv_case1.csv").
    withColumn("timeId", monotonically_increasing_id.cast(LongType)).
    withColumn("patId", lit(1).cast(LongType));
cases.cache()

[_c0: double, _c1: double ... 576 more fields]

In [4]:
val caseNumbers = Array(2,8,9,10,11,12)
/*loop through the rest of the files and append them to the first one*/
for(num <- caseNumbers){
    var caseTemp = spark.read.option("header",false).
            option("inferSchema",true).
            option("nanValue","0").
            option("nullValue","0").
            option("positiveInf","0").
            option("negativeInf","0").
            /*schema(schema).*/
            csv("swift2d://MGH." + name + "/csv/csv_case"+num.toString+".csv").
            withColumn("timeId", monotonically_increasing_id.cast(LongType)).
            withColumn("patId", lit(num).cast(LongType))
    cases = cases.union(caseTemp)
}
cases = cases.repartition(200)

In [5]:
/* files that had Inf need to have the inf changed to 0 and made into a decimal type */
for(col <- cases.schema){
    if(col.dataType == StringType){
        cases = cases.withColumn(col.name + "temp", 
                                 regexp_replace(
                                     regexp_replace(cases(col.name),"Inf","0")
                                     ,"NaN","0").cast(DoubleType))
        cases = cases.drop(col.name).withColumnRenamed(col.name + "temp",col.name)
    }
}
/* replace NaN with 0*/
cases = cases.na.fill(0)

In [6]:
println(cases.count())

247625


In [7]:
/*features are all columns exept patient id and timeid*/
val feature_cols = cases.columns.filter(x => (x != "timeId" && x != "patId"))
val assembler = new VectorAssembler().setInputCols(feature_cols).setOutputCol("features")
cases = assembler.transform(cases)

In [8]:
/*Normalize the features*/
val normalizer = new Normalizer().setInputCol("features").setOutputCol("normFeatures")
cases = normalizer.transform(cases)

In [9]:
/*pca with k = 100*/
val pcaModel = new PCA().setInputCol("normFeatures").setOutputCol("pcaFeatures").setK(100).fit(cases)

cases = pcaModel.transform(cases)
cases = cases.cache()

In [10]:
cases.limit(2).collect().foreach(println)

[2.4733,2.187,2.4774,2.3016,2.4757,2.1608,2.4887,2.2648,2.4914,2.1721,2.5604,2.2763,2.7022,2.2484,2.5371,2.5298,4.2015,3.8457,213060.0,220270.0,241640.0,274030.0,98981.0,106550.0,116250.0,125280.0,46622.0,48622.0,55668.0,57860.0,13812.0,13039.0,13170.0,17232.0,69.24,61.978,79.84,78.646,65.074,60.964,75.122,70.375,61.882,53.938,72.924,64.03,84.283,56.773,75.319,88.017,132.26,108.55,163.95,173.94,122.22,101.13,143.77,142.43,114.28,84.163,132.53,120.44,167.41,89.218,144.75,172.18,0.80077,0.87416,0.84238,0.87572,0.78706,0.88262,0.83856,0.87323,0.79206,0.88542,0.83324,0.87601,0.79585,0.89057,0.85261,0.89242,0.23412,0.15662,0.19285,0.14576,0.25227,0.14119,0.19483,0.15084,0.24313,0.15237,0.20481,0.14456,0.25049,0.13447,0.17184,0.12028,0.038331,0.028779,0.0355,0.030464,0.042366,0.031673,0.035687,0.029502,0.042562,0.031594,0.04341,0.034158,0.033546,0.033366,0.051541,0.038986,0.012127,0.008647,0.010865,0.007221,0.014185,0.0098834,0.011079,0.0078552,0.01705,0.01021,0.013334,0.0094176,0.015117,0.0

In [9]:
/*saving to parquet to be labeled with t-sne coordinates*/
/*cases.write.parquet("swift2d://MGH." + name + "/tempParq/7cases.parquet")*/

### 15 < k < 25

k: 2
Within Set Sum of Squared Errors = 2.9112884730548816E16
k: 4
Within Set Sum of Squared Errors = 1.7992007329050206E16
k: 6
Within Set Sum of Squared Errors = 1.4551705278862694E16
k: 8
Within Set Sum of Squared Errors = 1.2885932983311728E16
k: 10
Within Set Sum of Squared Errors = 1.11832826573762E16
k: 12
Within Set Sum of Squared Errors = 9.081206983606092E15
k: 14
Within Set Sum of Squared Errors = 8.522317069652645E15
k: 16
Within Set Sum of Squared Errors = 7.432428413580828E15
k: 18
Within Set Sum of Squared Errors = 7.576234664949629E15
k: 20
Within Set Sum of Squared Errors = 6.743778287620577E15
k: 22
Within Set Sum of Squared Errors = 6.43607511491788E15
k: 24
Within Set Sum of Squared Errors = 5.954476368201517E15
k: 26
Within Set Sum of Squared Errors = 5.743758395644838E15
k: 28
Within Set Sum of Squared Errors = 5.479222949405443E15
k: 30
Within Set Sum of Squared Errors = 5.348950188771225E15
k: 32
Within Set Sum of Squared Errors = 4.954711625236001E15
k: 34
Within Set Sum of Squared Errors = 4.835574388374279E15
k: 36
Within Set Sum of Squared Errors = 4.815777105637914E15
k: 38
Within Set Sum of Squared Errors = 4.724896615554858E15
k: 40
Within Set Sum of Squared Errors = 4.611871574389138E15

In [20]:
/*kmeans with wsse.  A good k could be between 15 and 25*/
/*for(a <- 10 to 30){
    println( "k: " + a )
    var kmeans = new KMeans().setFeaturesCol("pcaFeatures").setK(a)
    var model = kmeans.fit(cases)

    var WSSSE = model.computeCost(cases)
    println(s"Within Set Sum of Squared Errors = $WSSSE")
}*/

k: 10
Within Set Sum of Squared Errors = 9.541751238917884E15
k: 11
Within Set Sum of Squared Errors = 9.405817615503538E15
k: 12
Within Set Sum of Squared Errors = 8.851807434347111E15
k: 13
Within Set Sum of Squared Errors = 8.802021290934255E15
k: 14
Within Set Sum of Squared Errors = 8.004543550440997E15
k: 15
Within Set Sum of Squared Errors = 8.693538279624738E15
k: 16
Within Set Sum of Squared Errors = 7.525560733484382E15
k: 17
Within Set Sum of Squared Errors = 7.402581031959187E15
k: 18
Within Set Sum of Squared Errors = 7.085525522699249E15
k: 19
Within Set Sum of Squared Errors = 6.946290508784244E15
k: 20
Within Set Sum of Squared Errors = 6.930587732736565E15
k: 21
Within Set Sum of Squared Errors = 6.649059351596769E15
k: 22
Within Set Sum of Squared Errors = 6.354151882042983E15
k: 23
Within Set Sum of Squared Errors = 6.179099280347436E15
k: 24
Within Set Sum of Squared Errors = 5.931319774884071E15
k: 25
Within Set Sum of Squared Errors = 5.633257710434543E15
k: 26
Wi

In [11]:
/*with 18 clusters, label all points*/
var kmeans = new KMeans().setFeaturesCol("pcaFeatures").setK(18)
var kmeansModel = kmeans.fit(cases)
cases = kmeansModel.transform(cases)

In [15]:
cases.where(cases("timeId") === 500).select("prediction","timeId","patId","pcaFeatures").limit(2).collect().foreach(println)

[9,500,2,[-0.8913514635526217,0.28184214160574783,0.09484615459373412,0.03932321831211063,0.3261817326804047,0.06875527761626192,0.048190292742114206,-0.02671829641724971,-0.015194883522910829,-0.0022561275651611173,0.023712576427751094,-0.017159175229942786,0.004858098733190709,-0.008210395230528812,2.019005307234183E-4,0.009453815525119315,0.0023100174647202585,-0.003399285832409724,-2.4873041847222685E-5,8.594988493730671E-4,0.006912573893086314,0.0011790676212299375,9.705696647936224E-4,-9.343408192160656E-4,0.001082671331772317,-0.003191923582872895,-0.0013481837750263688,-9.125377491055172E-4,-5.673014089510171E-4,1.4478136430241855E-4,3.988441497400934E-5,-5.344184143215593E-4,-1.1872176727981114E-4,-3.31902624669915E-4,3.0637773584464226E-4,2.0723826825522033E-4,-4.7502386137230704E-5,2.34140835627927E-5,-2.7168596385520775E-4,5.870480396553255E-5,-1.0038014032166838E-5,3.004995797187692E-5,-1.9545698355488882E-5,3.8079769309312504E-4,3.850980669913168E-4,5.816496628783086E-4,1

In [16]:
cases.write.parquet("swift2d://MGH." + name + "/tempParq/7cases.parquet")

1


In [15]:
/*if we pce with k = 2, we can use coordinates to visualize*/
/*this is run seperately from the above code*/
val twoPcaModel = new PCA().setInputCol("pcaFeatures").setOutputCol("twoFeat").setK(2).fit(cases)
var kmeansVis = twoPcaModel.transform(cases).select("twoFeat","prediction","timeId","patId")

In [51]:
kmeansVis.limit(2).collect().foreach(println)

[[103791.16920407301,-5769.470836937589],2]
[[102351.29636902601,-5238.4915275905],2]


In [21]:
var badData = spark.read.option("header",false).
    option("inferSchema",true).
    option("nanValue","0").
    option("nullValue","0").
    option("positiveInf","0").
    option("negativeInf","0").
    /*schema(schema).*/
    csv("swift2d://MGH." + name + "/csv/csv_case3.csv").
    withColumn("timeId", monotonically_increasing_id.cast(LongType)).
    withColumn("patId", lit(1).cast(LongType));
badData.printSchema

1
root
 |-- _c0: double (nullable = true)
 |-- _c1: double (nullable = true)
 |-- _c2: double (nullable = true)
 |-- _c3: double (nullable = true)
 |-- _c4: double (nullable = true)
 |-- _c5: double (nullable = true)
 |-- _c6: double (nullable = true)
 |-- _c7: double (nullable = true)
 |-- _c8: double (nullable = true)
 |-- _c9: double (nullable = true)
 |-- _c10: double (nullable = true)
 |-- _c11: double (nullable = true)
 |-- _c12: double (nullable = true)
 |-- _c13: double (nullable = true)
 |-- _c14: double (nullable = true)
 |-- _c15: double (nullable = true)
 |-- _c16: double (nullable = true)
 |-- _c17: double (nullable = true)
 |-- _c18: double (nullable = true)
 |-- _c19: double (nullable = true)
 |-- _c20: double (nullable = true)
 |-- _c21: double (nullable = true)
 |-- _c22: double (nullable = true)
 |-- _c23: double (nullable = true)
 |-- _c24: double (nullable = true)
 |-- _c25: double (nullable = true)
 |-- _c26: double (nullable = true)
 |-- _c27: double (nullable = t

In [23]:
/*create schema*/
import scala.collection.mutable.ArrayBuffer
var fields = ArrayBuffer[StructField]()
for(num <- 0 to 575){
    fields += StructField("_c" + num.toString,DecimalType(10,0))
}
var schema = StructType(fields)
schema.fields

Array(StructField(_c0,DecimalType(10,0),true), StructField(_c1,DecimalType(10,0),true), StructField(_c2,DecimalType(10,0),true), StructField(_c3,DecimalType(10,0),true), StructField(_c4,DecimalType(10,0),true), StructField(_c5,DecimalType(10,0),true), StructField(_c6,DecimalType(10,0),true), StructField(_c7,DecimalType(10,0),true), StructField(_c8,DecimalType(10,0),true), StructField(_c9,DecimalType(10,0),true), StructField(_c10,DecimalType(10,0),true), StructField(_c11,DecimalType(10,0),true), StructField(_c12,DecimalType(10,0),true), StructField(_c13,DecimalType(10,0),true), StructField(_c14,DecimalType(10,0),true), StructField(_c15,DecimalType(10,0),true), StructField(_c16,DecimalType(10,0),true), StructField(_c17,DecimalType(10,...

In [10]:
/* all double columns need to be made to decimal*/
import java.util.Calendar
println(Calendar.getInstance().getTime())
println(now)
for(col <- cases.schema){
    println(col.name)
    if(col.dataType == DoubleType){
        cases = cases.withColumn(col.name + "temp", cases(col.name).cast(DecimalType(10,0))).drop(col.name).withColumnRenamed(col.name + "temp",col.name)
    }
}
println(Calendar.getInstance().getTime())

Wed May 31 21:05:43 CDT 2017
Wed May 31 21:04:38 CDT 2017
_c0
_c1
_c2
_c3
_c4
_c5
_c6
_c7
_c8
_c9
_c10
_c11
_c12
_c13
_c14
_c15
_c30
_c31
_c32
_c33
_c34
_c35
_c36
_c37
_c38
_c39
_c40
_c41
_c42
_c43
_c44
_c45
_c46
_c47
_c48
_c49
_c50
_c51
_c52
_c53
_c54
_c55
_c56
_c57
_c58
_c59
_c60
_c61
_c62
_c63
_c64
_c65
_c66
_c67
_c68
_c69
_c70
_c71
_c72
_c73
_c74
_c75
_c76
_c77
_c78
_c79
_c80
_c81
_c82
_c83
_c84
_c85
_c86
_c87
_c88
_c89
_c90
_c91
_c92
_c93
_c94
_c95
_c96
_c97
_c98
_c99
_c100
_c101
_c102
_c103
_c104
_c105
_c106
_c107
_c108
_c109
_c110
_c111
_c112
_c113
_c114
_c115
_c116
_c117
_c118
_c119
_c120
_c121
_c122
_c123
_c124
_c125
_c126
_c127
_c128
_c129
_c130
_c131
_c132
_c133
_c134
_c135
_c136
_c137
_c138
_c139
_c140
_c141
_c142
_c143
_c144
_c145
_c146
_c147
_c148
_c149
_c150
_c151
_c152
_c153
_c154
_c155
_c156
_c157
_c158
_c159
_c160
_c161
_c162
_c163
_c164
_c165
_c166
_c167
_c168
_c169
_c170
_c171
_c172
_c173
_c174
_c175
_c176
_c177
_c178
_c179
_c180
_c181
_c182
_c183
_c184
_c185
_c186


Adding a schema didn't work for nan/inf values